In [ ]:
# centroids generated from this step do not go through the pixel filter!
# but centroids from first half were generated via this step soooooooooooooooooo
# I could generate some centroids from the first half and compare to current values
# looks like I already generated those values
#so why is second half getting flagged? differently
#doesnt explain why there are half differences but does explain the fact the flower is getting picked up p frequ. 
#one filter only saves centroid values for objects with a contour larger than x
#the other filter saves in and out values for 

In [2]:
import time
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os, errno
import glob

In [7]:
def FrameSequence(file_name, path):
    video = [(os.path.join(root, file), file[0:-4]) 
        for root, dirs, files in os.walk(path) 
        for file in files 
        if file.endswith(file_name + '.mp4')]
    cam = cv2.VideoCapture(video[0][0])
    cam.set(1,0)
    frame_sequence = list(range(0,int(cam.get(cv2.CAP_PROP_FRAME_COUNT))))

    return frame_sequence

In [15]:
file_truet = r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Step6\AllLight_EveryMoth.csv"
location = r'C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Step1_Tanvi_Method'
outpath = r'C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Motion_Analysis\Centroids'
video = r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Videos\All_videos_used_for_analysis"

In [ ]:
# open data frame with all true trial info
# pull out each video based on a name from a list
# for each name pull out all start and end trials that also have true trials
# choose the start and end frame for the first and last entry
# use those integers to loop through (one loop for first centroid and one for last centroid - seperate cells to generate seperate csv w/o altering the visit type)
#     end_frame: list_start/finish[len(list_start/finish)-1]

In [16]:
dFrame = pd.read_csv(file_truet)
list_videos = dFrame.name[dFrame.name.str.contains('L')].unique()

for name in list_videos:
    i = 0
    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)
    print(len(In_frame), name)

4 L0.1_c-3_m10
3 L0.1_c-3_m12
63 L0.1_c-3_m20
63 L0.1_c-3_m21
20 L0.1_c-3_m22
7 L0.1_c-3_m23
1 L0.1_c-3_m24
2 L0.1_c-3_m25
9 L0.1_c-3_m27
10 L0.1_c-3_m2
20 L0.1_c-3_m32
26 L0.1_c-3_m34
7 L0.1_c-3_m37
13 L0.1_c-3_m38
14 L0.1_c-3_m39
25 L0.1_c-3_m40
15 L0.1_c-3_m41
12 L0.1_c-3_m43
34 L0.1_c-3_m44
38 L0.1_c-3_m5
22 L0.1_c-3_m8
8 L50_c-3_m10
25 L50_c-3_m12
22 L50_c-3_m13
14 L50_c-3_m14
24 L50_c-3_m15
25 L50_c-3_m21
49 L50_c-3_m22
1 L50_c-3_m24
1 L50_c-3_m25
11 L50_c-3_m26
2 L50_c-3_m2
3 L50_c-3_m30
33 L50_c-3_m32
3 L50_c-3_m33
4 L50_c-3_m34
9 L50_c-3_m35
11 L50_c-3_m37
8 L50_c-3_m38
32 L50_c-3_m39
14 L50_c-3_m45
8 L50_c-3_m6
8 L50_c-3_m9


In [14]:
dFrame = pd.read_csv(file_truet)
list_videos = dFrame.name[dFrame.name.str.contains('L')].unique()
list_videos

array(['L0.1_c-3_m10', 'L0.1_c-3_m12', 'L0.1_c-3_m20', 'L0.1_c-3_m21',
       'L0.1_c-3_m22', 'L0.1_c-3_m23', 'L0.1_c-3_m24', 'L0.1_c-3_m25',
       'L0.1_c-3_m27', 'L0.1_c-3_m2', 'L0.1_c-3_m32', 'L0.1_c-3_m34',
       'L0.1_c-3_m37', 'L0.1_c-3_m38', 'L0.1_c-3_m39', 'L0.1_c-3_m40',
       'L0.1_c-3_m41', 'L0.1_c-3_m43', 'L0.1_c-3_m44', 'L0.1_c-3_m5',
       'L0.1_c-3_m8', 'L50_c-3_m10', 'L50_c-3_m12', 'L50_c-3_m13',
       'L50_c-3_m14', 'L50_c-3_m15', 'L50_c-3_m21', 'L50_c-3_m22',
       'L50_c-3_m24', 'L50_c-3_m25', 'L50_c-3_m26', 'L50_c-3_m2',
       'L50_c-3_m30', 'L50_c-3_m32', 'L50_c-3_m33', 'L50_c-3_m34',
       'L50_c-3_m35', 'L50_c-3_m37', 'L50_c-3_m38', 'L50_c-3_m39',
       'L50_c-3_m45', 'L50_c-3_m6', 'L50_c-3_m9'], dtype=object)

In [41]:
for name in list_videos:

    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)
    print(len(In_frame), name)
    
    print(name)
    frames = FrameSequence(name, video)

    start_frame = In_frame[0]-5
    end_frame = Out_frame[0]+5
    frameNum = frames[start_frame:end_frame]
    contours = []
    centroids_x = []
    centroids_y = []
    contours_hull = []
    centroids_hull_x = []
    centroids_hull_y = []
    frame_ = []

    df  = pd.read_csv(r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Step1_Tanvi_Method" + "\\" + name + ".csv")

    
    for frame in frameNum:
        cent_x = list(df[(df.index == frame)].Centroid_hull_x)
        cent_y = list(df[(df.index == frame)].Centroid_hull_y)
        centroids_hull_x.append(np.squeeze(np.asarray(cent_x)))
        centroids_hull_y.append(np.squeeze(np.asarray(cent_y)))
        frame_.append(frame)

    new_df = (pd.DataFrame({'FrameNumber' : frame_, 'Name': name, 'X_position' : centroids_hull_x, 
                                'Y_position' : centroids_hull_y}))

    new_df.to_csv(outpath + "//" + 'Final_position_centroid_method_FirstTrial_' + name + '.csv')

38 L0.1_c-3_m5
L0.1_c-3_m5


In [29]:
second_last = ['L0.1_c-3_m10', 'L0.1_c-3_m12', 'L0.1_c-3_m20', 'L0.1_c-3_m21',
       'L0.1_c-3_m22', 'L0.1_c-3_m23', 'L0.1_c-3_m24', 'L0.1_c-3_m25',
       'L0.1_c-3_m27', 'L0.1_c-3_m2', 'L0.1_c-3_m32', 'L0.1_c-3_m34',
       'L0.1_c-3_m37', 'L0.1_c-3_m38', 'L0.1_c-3_m39', 'L0.1_c-3_m40',
       'L0.1_c-3_m41', 'L0.1_c-3_m43', 'L0.1_c-3_m44', 'L0.1_c-3_m5',
       'L0.1_c-3_m8', 'L50_c-3_m10', 'L50_c-3_m12', 'L50_c-3_m13',
       'L50_c-3_m14', 'L50_c-3_m15', 'L50_c-3_m21', 'L50_c-3_m22',
       'L50_c-3_m24', 'L50_c-3_m25', 'L50_c-3_m26', 'L50_c-3_m2',
       'L50_c-3_m30', 'L50_c-3_m32', 'L50_c-3_m33', 'L50_c-3_m34',
       'L50_c-3_m35', 'L50_c-3_m37', 'L50_c-3_m38', 'L50_c-3_m39',
       'L50_c-3_m45', 'L50_c-3_m6', 'L50_c-3_m9']

In [ ]:
second_last = [('L0.1_c-3_m10', -2), ('L0.1_c-3_m12', -2), ('L0.1_c-3_m20', -1), ('L0.1_c-3_m21', -1),
       ('L0.1_c-3_m22', -1), ('L0.1_c-3_m23', -2), ('L0.1_c-3_m25', -1), (
       'L0.1_c-3_m27', -5), ('L0.1_c-3_m2', -1), ('L0.1_c-3_m32', -1), ('L0.1_c-3_m34', -1),
       ('L0.1_c-3_m37', 2), ('L0.1_c-3_m38', -1), ('L0.1_c-3_m40', -2),
       ('L0.1_c-3_m41', -2), ('L0.1_c-3_m43', -1), ('L0.1_c-3_m44', -1), ('L0.1_c-3_m5', -1),
       ('L0.1_c-3_m8', 11), ('L50_c-3_m10', -1), ('L50_c-3_m12', -3), ('L50_c-3_m13', -1),
       ('L50_c-3_m14', -1), ('L50_c-3_m15', -1), ('L50_c-3_m21', -1), ('L50_c-3_m22', -1),
       ('L50_c-3_m26', -1), ('L50_c-3_m2', -1),
       ('L50_c-3_m30', -2), ('L50_c-3_m32', -1), ('L50_c-3_m33', -1), ('L50_c-3_m34', -1),
       ('L50_c-3_m35', -4), ('L50_c-3_m37', -2), ('L50_c-3_m38', 2),  ('L50_c-3_m39', -1),
       ('L50_c-3_m45', -9), ('L50_c-3_m6', -1), ('L50_c-3_m9', -1)]

In [45]:
revamp = [('L0.1_c-3_m34', -2), ('L50_c-3_m38', 1)]

In [46]:
for name in revamp:
    index = name[1]
    name = name[0]
    
    In_frame = list(dFrame[(dFrame.name == name) & (dFrame.In_Frame.notnull())].In_Frame)
    
    Out_frame = list(dFrame[(dFrame.name == name) & (dFrame.Out_Frame.notnull())].Out_Frame)


    print(name)
    frames = FrameSequence(name, video)

    start_frame = In_frame[index]+20
    end_frame = Out_frame[index]-20
    frameNum = frames[start_frame:end_frame]
    contours = []
    centroids_x = []
    centroids_y = []
    contours_hull = []
    centroids_hull_x = []
    centroids_hull_y = []
    frame_ = []

    
    df  = pd.read_csv(r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Step1_Tanvi_Method" + "\\" + name + ".csv")

    
    for frame in frameNum:
        cent_x = list(df[(df.index == frame)].Centroid_hull_x)
        cent_y = list(df[(df.index == frame)].Centroid_hull_y)
        centroids_hull_x.append(np.squeeze(np.asarray(cent_x)))
        centroids_hull_y.append(np.squeeze(np.asarray(cent_y)))
        frame_.append(frame)
    new_df = (pd.DataFrame({'FrameNumber' : frame_, 'Name': name, 'X_position' : centroids_hull_x, 
                                'Y_position' : centroids_hull_y}))
    second_last_outpath = r"C:\Users\Daniellab\Desktop\Light_level_videos_second_batch\Data\Motion_Analysis\First_Last_Centroids\Second_Last"
    new_df.to_csv(outpath + "//" + 'Final_position_centroid_method_Last_SecondTrial_' + name + '.csv')

L0.1_c-3_m34
L50_c-3_m38
